In [3]:
print("hello world")

hello world


In [2]:
import tensorflow as tf
import os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

from keras.preprocessing.image import ImageDataGenerator
import numpy as np

inputs = Input(shape=(224, 224, 3))
# Building a pretrained VGG-16 feature extractor as encoder:
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
# We recover the feature maps returned by each of the 3 final blocks:
f3 = vgg16.get_layer('block3_pool').output # shape: (28, 28, 256)
f4 = vgg16.get_layer('block4_pool').output # shape: (14, 14, 512)
f5 = vgg16.get_layer('block5_pool').output # shape: ( 7, 7, 512)
# We replace the VGG dense layers by convs, adding the "decoding" layers
instead after the conv/pooling blocks:
f3 = Conv2D(filters=out_ch, kernel_size=1, padding='same')(f3)
f4 = Conv2D(filters=out_ch, kernel_size=1, padding='same')(f4)
f5 = Conv2D(filters=out_ch, kernel_size=1, padding='same')(f5)
# We upscale `f5` to a 14x14 map so it can be merged with `f4`:
f5x2 = Conv2DTranspose(filters=out_chh, kernel_size=4,strides=2,
padding='same', activation='relu')(f5)
# We merge the 2 feature maps with an element-wise addition:
m1 = add([f4, f5x2])
# We repeat the operation to merge `m1` and `f3` into a 28x28 map:
m1x2 = Conv2DTranspose(filters=out_ch, kernel_size=4, strides=2,
padding='same', activation='relu')(m1)
m2 = add([f3, m1x2])
# Finally, we use a transp-conv to recover the original shape:
outputs = Conv2DTranspose(filters=out_ch, kernel_size=16, strides=8,
padding='same', activation='sigmoid')(m2)
fcn_8s = Model(inputs, outputs)

In [4]:
model.add(tf.keras.layers.Dense(128, activation='relu'))

In [10]:
data ="""Once your text is split into tokens, you need to encode each token into a numerical
representation. You could potentially do this in a stateless way, such as by hashing each
token into a fixed binary vector, but in practice, the way you'd go about it is to build
an index of all terms found in the training data (the “vocabulary”), and assign a
unique integer to each entry in the vocabulary.
Something like this"""
data = data.replace("!", "")
data = data.replace("?", "")
data = data.lower().split()
print(data)

vocabulary = {}
for word in data:
    if word not in vocabulary:
        vocabulary[word] = len(vocabulary)

print(vocabulary)

['once', 'your', 'text', 'is', 'split', 'into', 'tokens,', 'you', 'need', 'to', 'encode', 'each', 'token', 'into', 'a', 'numerical', 'representation.', 'you', 'could', 'potentially', 'do', 'this', 'in', 'a', 'stateless', 'way,', 'such', 'as', 'by', 'hashing', 'each', 'token', 'into', 'a', 'fixed', 'binary', 'vector,', 'but', 'in', 'practice,', 'the', 'way', "you'd", 'go', 'about', 'it', 'is', 'to', 'build', 'an', 'index', 'of', 'all', 'terms', 'found', 'in', 'the', 'training', 'data', '(the', '“vocabulary”),', 'and', 'assign', 'a', 'unique', 'integer', 'to', 'each', 'entry', 'in', 'the', 'vocabulary.', 'something', 'like', 'this']
{'once': 0, 'your': 1, 'text': 2, 'is': 3, 'split': 4, 'into': 5, 'tokens,': 6, 'you': 7, 'need': 8, 'to': 9, 'encode': 10, 'each': 11, 'token': 12, 'a': 13, 'numerical': 14, 'representation.': 15, 'could': 16, 'potentially': 17, 'do': 18, 'this': 19, 'in': 20, 'stateless': 21, 'way,': 22, 'such': 23, 'as': 24, 'by': 25, 'hashing': 26, 'fixed': 27, 'binary': 

In [38]:
import numpy as np
def one_hot_encode_token(token):
    vector = np.zeros((len(vocabulary),))
    token_index = vocabulary[token]
    vector[token_index] = 1
    return vector


z = np.zeros((len(vocabulary),))
for key in vocabulary.keys():
    # z = np.hstack((z, one_hot_encode_token(key)))
    print(key)
print(z)


once
your
text
is
split
into
tokens,
you
need
to
encode
each
token
a
numerical
representation.
could
potentially
do
this
in
stateless
way,
such
as
by
hashing
fixed
binary
vector,
but
practice,
the
way
you'd
go
about
it
build
an
index
of
all
terms
found
training
data
(the
“vocabulary”),
and
assign
unique
integer
entry
vocabulary.
something
like
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [33]:
print(z)

[0. 0. 0. ... 0. 0. 1.]


In [36]:
z.size

3306

In [49]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(output_mode="int",)

import re
import string
import tensorflow as tf
def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(lowercase_string, f"[{re.escape(string.punctuation)}]", "")
def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)
text_vectorization = TextVectorization(output_mode="int",standardize=custom_standardization_fn, split=custom_split_fn,)

In [50]:
dataset = [
"I write, erase, rewrite",
"Erase again, and then",
"A poppy blooms.",
]
text_vectorization.adapt(dataset)

In [51]:
text_vectorization.get_vocabulary()

['', '[UNK]']

In [47]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [48]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [13]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
"aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
"aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
"aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [14]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break


inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'Big S isn\'t playing with taboos or forcing an agenda like, say Mencia or Chapelle (though I like them both). She states the obvious in subtle, near subliminal remarks. Her show won\'t change the World, nor is it meant to. But, along with the hilarious Brian Posehn and Paget Brewster\'s ex-boyfriend Jay Johnston of "Mr. Show" fame, this is one mean show with an appetite for destruction! My side\'s were thoroughly wrecked by the first episode. Look, I love this woman and like her famed boyfriend, Jimmy Kimmel, she just delivers the lines and lets the viewer run- with-it. The best kind of comedy around. Spoofing anything and anyone, like "Mary Poppins" in the second episode when she sings to the fake birds on to quick hitting commentary on society and college aged existential nonsense. This one is highly recommended, but only for those who still have a funny bon

In [15]:
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
        ngrams=2,
        max_tokens=20000,
        output_mode="tf_idf",
)

In [16]:
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

In [17]:
from tensorflow import keras
from tensorflow.keras import layers
def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                    loss="binary_crossentropy",
                    metrics=["accuracy"])
    return model

In [18]:

tfidf_2gram_train_ds = train_ds.map(
                lambda x, y: (text_vectorization(x), y),
                num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
                lambda x, y: (text_vectorization(x), y),
                num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
                lambda x, y: (text_vectorization(x), y),
                num_parallel_calls=4)

In [19]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [20]:
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",save_best_only=True)
]

In [21]:
model.fit(
    tfidf_2gram_train_ds.cache(),
    validation_data=tfidf_2gram_val_ds.cache(),
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
625/625 [==============================] - 20s 28ms/step - loss: 0.4881 - accuracy: 0.7987 - val_loss: 0.3110 - val_accuracy: 0.8902
Epoch 2/10
625/625 [==============================] - 13s 20ms/step - loss: 0.3133 - accuracy: 0.8783 - val_loss: 0.3007 - val_accuracy: 0.8952
Epoch 3/10
625/625 [==============================] - 12s 20ms/step - loss: 0.2730 - accuracy: 0.8945 - val_loss: 0.3188 - val_accuracy: 0.8948
Epoch 4/10
625/625 [==============================] - 12s 19ms/step - loss: 0.2507 - accuracy: 0.9039 - val_loss: 0.3424 - val_accuracy: 0.8826
Epoch 5/10
625/625 [==============================] - 12s 19ms/step - loss: 0.2414 - accuracy: 0.9051 - val_loss: 0.3612 - val_accuracy: 0.8820
Epoch 6/10
625/625 [==============================] - 13s 20ms/step - loss: 0.2399 - accuracy: 0.9094 - val_loss: 0.3767 - val_accuracy: 0.8798
Epoch 7/10
625/625 [==============================] - 13s 21ms/step - loss: 0.2293 - accuracy: 0.9125 - val_loss: 0.3736 - val_accuracy:

In [22]:
model = keras.models.load_model("binary_1gram.keras")

print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 8s 10ms/step - loss: 0.2890 - accuracy: 0.8973
Test acc: 0.897
